### User OGS game data dump

**This notebook should be run first before any other utilities, as other utilities depend on the game data dump generated by this tool.**

This notebook generates a raw .json file of game information for games played with your OGS account.
It only tracks information *about* each game, and does not dump the games themselves.

### Dev notes:
There are roughly two ways of implementing what's happening in this notebook:
1. We could call the `https://online-go.com/api/v1/players/<player_id>/games` endpoint in the OGS API for a specified `player_id`.
2. We could find all the game id numbers for a given player (via method 1), then call the `https://online-go.com/api/v1/games/<game_id>` endpoint repeatedly for each game id.

In this notebook we choose method 1. In principal the endpoint in method 2 contains more detailed information about each individual game, including a JSON representation of the actual moves. However, method 1 involves fewer API requests as it collects data on 10 games per request, while method 2 requests a single game at a time.

Method 2 is implicitly invoked when running `games_dump.ipynb` as the game moves can only be accessed through the `https://online-go.com/api/v1/games/<game_id>` endpoint. A more detailed raw stats dump is performed there in case it is useful.

In [ ]:
import requests
import json
import time
import random
import os

Replace the following `player_id` number with the one whose games you wish to fetch.

In [ ]:
player_id = dummy_id

In [ ]:
# Obtain API authorization keys
with open('keys.txt', 'r') as f:
    lines = f.readlines()
    client_id = lines[0].split('\"')[1]
    client_secret = lines[1].split('\"')[1]
    username = lines[2].split('\"')[1]
    password = lines[3].split('\"')[1]

# Obtaining authorization token
s = requests.Session()
response = s.post('https://online-go.com/oauth2/token/',data={'grant_type':'password','username':username,'password':password,'client_id':client_id,'client_secret':client_secret})
result = response.json()
token = result['access_token']
headers = {'Authorization': 'Bearer {}'.format(token)}

The following cell fetches information for the player's games. The OGS API has a rate limit on requests, so there is also a very primitive rate backoff implemented. You will see some hangups in the output as the rate limit is encountered and the loop has to wait out its backoff time. This code block could take several minutes depending on how many games you've played.

In [ ]:
# Set endpoint, request first page of results
endpoint = 'https://online-go.com/api/v1/players/'+str(player_id)+'/games'
gamesPage = s.get(endpoint, headers = headers).json()
games = gamesPage['results']
nextPage = gamesPage['next']

attempts = 0
max_attempts = 10
while nextPage != None:
    response = s.get(nextPage,  headers = headers)
    # If not rate limited, continue
    if response.status_code != 429:
        attempts = 0
        print(nextPage)
        gamesPage = response.json()
        games.extend(gamesPage['results'])
        nextPage = gamesPage['next']
    # If rate limited, wait and then start again
    else:
        print('Rate limit reached. Backing off before retrying query. Please wait...')
        attempts += 1
        if attempts >= max_attempts:
            print('Too many attempts.')
            break
        else:
            time.sleep((2 ** attempts) + random.random())

The final cell creates a JSON file named `raw_stats.json` in the `./output/<player_id>/datasets` directory containing the raw data dump of the player's games.

In [ ]:
try:
    os.mkdir('output')
except FileExistsError:
    pass
try:
    os.mkdir('output/'+str(player_id))
except FileExistsError:
    pass
try:
    os.mkdir('output/'+str(player_id)+'/datasets')
except FileExistsError:
    pass

with open('./output/'+str(player_id)+'/datasets/raw_stats.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(games, indent = 2))